<a href="https://colab.research.google.com/github/bigdatavik/GDSClasss/blob/main/Retail_Demo_with_the_Graph_Data_Science_Library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retail Demo




In [1]:
# INSTALL CODE DEPENDENCIES and RESTART your RUNTIME
!pip install --upgrade --force-reinstall neo4j pandas matplotlib

     |████████████████████████████████| 71kB 4.3MB/s 
     |████████████████████████████████| 9.9MB 6.8MB/s 
     |████████████████████████████████| 10.3MB 40.9MB/s 
     |████████████████████████████████| 512kB 33.5MB/s 
     |████████████████████████████████| 15.3MB 333kB/s 
     |████████████████████████████████| 235kB 48.2MB/s 
     |████████████████████████████████| 2.2MB 40.7MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 1.1MB 40.3MB/s 
  Created wheel for neo4j: filename=neo4j-4.2.1-cp37-none-any.whl size=95190 sha256=1a45816362f7c7898c6f90665421687875bafc9b89a178b35761366ac5c6b7ee
  Stored in directory: /root/.cache/pip/wheels/00/a8/69/083e04c2ffc0a7e026c9356cd5bb54c65128f3e08ef8371be0
Successfully built neo4j
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.3 w

In [ ]:
from neo4j import GraphDatabase

# tag::imports[]
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# end::imports[]

In [ ]:
#bolt_uri = "bolt://link-prediction-neo4j"
bolt_uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(bolt_uri, auth=("neo4j", "letmein"))

print(driver.address)

ServiceUnavailable: ignored

### DB Schema

In [ ]:
query = """
CALL db.schema.visualization;
"""

with driver.session(database="neo4j") as session:
    #display(session.run(query).consume().counters)
    result = session.run(query)
    results = pd.DataFrame([dict(record) for record in result])
results 

,nodes,relationships
0,"[(name, indexes, constraints), (name, indexes,...","[(), (), (), (), (), (), (), ()]"


In [ ]:
query = """
CALL apoc.meta.subGraph(
{excludes:['SIMILAR', 'SIMILAR_EMBEDDING', 'SIMILAR_EVERYTHING_EMBEDDING', 'SIMILAR_NODE2VEC_EMBEDDING', 'Similar']});
"""

with driver.session(database="neo4j") as session:
    #display(session.run(query).consume().counters)
    result = session.run(query)
    results = pd.DataFrame([dict(record) for record in result])
results 


,nodes,relationships
0,"[(name, count), (name, count), (name, count), ...","[(count, type, in, out), (count, type, in, out..."


![Screen%20Shot%202021-03-25%20at%2010.14.15%20AM.png](attachment:Screen%20Shot%202021-03-25%20at%2010.14.15%20AM.png)

### Clean up Demo

In [ ]:
run_queries = """

// Drop all models
CALL gds.beta.model.list()
YIELD modelInfo AS mInfo
WITH mInfo.modelName as savedModel
CALL gds.beta.model.drop(savedModel)
YIELD modelInfo
RETURN modelInfo;

// Drop all graphs
CALL gds.graph.list()
YIELD graphName AS namedGraph
WITH namedGraph
CALL gds.graph.drop(namedGraph)
YIELD graphName
RETURN graphName;

// REMOVE properties
MATCH (i:Item) REMOVE i.embedding;
MATCH (i:Item) REMOVE i.graphsage_embedding;
MATCH (i:Item) REMOVE i.node2vec_embedding;
MATCH (i:Item) REMOVE i.graphsage_embedding_everything;

// Delete extra OWLs
UNWIND ['SEA OWL', 'LAND OWL', 'FIRE OWL', 'FOREST OWL', 'POTATO OWL'] as descr
MATCH (i:Item {Description:descr})
DETACH DELETE i;

// REMOVE extra similarity rels
UNWIND ['SIMILAR_EMBEDDING', 'SIMILAR_GRAPHSAGE_EMBEDDING_MULTIPARTITE','SIMILAR_GRAPHSAGE_EMBEDDING','SIMILAR_GRAPHSAGE_EMBEDDING_KNN', 'SIMILAR_NODE2VEC_EMBEDDING',
      'SIMILAR_EVERYTHING_EMBEDDING', 'Similar', 'SIMILAR'] as rel
MATCH ()-[r]->() WHERE type(r) = rel
DELETE r;   
"""


with driver.session() as session:
    for statement in run_queries.split(';'):
        try:
            session.run(statement.strip())
        except:
            pass

### Load Item Affinity Graph

In [ ]:
query = """
CALL gds.graph.create(
  'items',
  {
    Item: {
      label: 'Item',
      properties: {
        price: {
          property: 'avg_price',
          defaultValue: 0.0
        },
        price_scaled: {
         property: 'avg_price_scaled',
         defaultValue: 0
       },
       total_sold: {
         property: 'num_sold',
         defaultValue: 0
       },
       total_sold_scaled: {
         property: 'num_sold_scaled',
         defaultValue: 0
       }
      }
    }
  },
  {
      copurchased: {
        type: 'COPURCHASED',
      orientation: 'UNDIRECTED'
    }
  }
);
"""

with driver.session(database="neo4j") as session:
    #display(session.run(query).consume().counters)
    result = session.run(query)
    results = pd.DataFrame([dict(record) for record in result])
results 

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,createMillis
0,{'Item': {'properties': {'price_scaled': {'pro...,"{'copurchased': {'orientation': 'UNDIRECTED', ...",items,2864,2430254,479


### **Node2Vec** on items Graph

In [ ]:
query = """
CALL gds.alpha.node2vec.stream('items', {
     embeddingDimension: 8, 
     walkLength:16, 
     returnFactor: 1.0,
     inOutFactor:.5,
     concurrency:4});
"""

with driver.session(database="neo4j") as session:
    #display(session.run(query).consume().counters)
    result = session.run(query)
    results = pd.DataFrame([dict(record) for record in result])
results 

,nodeId,embedding
0,12,"[-0.48127540946006775, 0.18506743013858795, 0...."
1,13,"[0.4862574338912964, 0.5233066082000732, 0.576..."
2,14,"[-0.05882648006081581, 0.3673568665981293, 0.2..."
3,15,"[-0.09630923718214035, 0.27312034368515015, 0...."
4,16,"[-0.2119910717010498, 0.3726838231086731, 0.33..."
...,...,...
2859,2871,"[-0.31048938632011414, 0.17524071037769318, 0...."
2860,2872,"[0.7993035912513733, -0.012966255657374859, -0..."
2861,2873,"[-0.12410412728786469, 0.3660004138946533, -0...."
2862,2874,"[-0.3239247798919678, 0.0857771709561348, 0.03..."


### **Node2Vec** write embeddings to graph

In [ ]:
query = """
CALL gds.alpha.node2vec.write('items', 
    {
     embeddingDimension: 8, 
     walkLength:16, 
     returnFactor: 1.0,
     inOutFactor:.5,
     writeProperty:'node2vec_embedding',
     concurrency:4
    }
)
"""

with driver.session(database="neo4j") as session:
    #display(session.run(query).consume().counters)
    result = session.run(query)
    results = pd.DataFrame([dict(record) for record in result])
results

,nodeCount,nodePropertiesWritten,createMillis,computeMillis,writeMillis,configuration
0,2864,2864,0,745185,370,"{'initialLearningRate': 0.025, 'writeConcurren..."


### **Calculate Similarity**

In [ ]:
query = """
MATCH (i:Item)
 WITH {item:id(i), weights: i.node2vec_embedding} AS itemData
 WITH collect(itemData) AS data
 CALL gds.alpha.similarity.cosine.write({
  data: data,
  skipValue: null,
  topK: 5,
  similarityCutoff:.1,
  writeRelationshipType:'SIMILAR_NODE2VEC_EMBEDDING'
 })
YIELD min, max, mean, stdDev, p25, p50, p75, p90, p95, p99, p999, p100
RETURN min, max, mean, stdDev;
"""

with driver.session(database="neo4j") as session:
    #display(session.run(query).consume().counters)
    result = session.run(query)
    results = pd.DataFrame([dict(record) for record in result])
results

,min,max,mean,stdDev
0,0.726,1.000,0.961,0.050


### **GraphSAGE** model training on items graph

In [ ]:
query = """
CALL gds.beta.graphSage.train(
  'items',
  {
    embeddingDimension: 16,
    modelName: 'graphsage_demo',
    featureProperties: ['total_sold_scaled', 'price_scaled'],
    aggregator: 'pool', //aggregating data
    activationFunction: 'sigmoid', //part of the aggregation function
    degreeAsProperty: true, //adding more properties
    epochs: 5, //how many times to traverse the graph during training
    searchDepth:5, //depth of the random walk
    sampleSizes:[25,10], //how many nodes to sample at each layer
    negativeSampleWeight: 10 //used by the loss function
  }
);
"""

with driver.session(database="neo4j") as session:
    #display(session.run(query).consume().counters)
    result = session.run(query)
    results = pd.DataFrame([dict(record) for record in result])
results 


,graphName,graphCreateConfig,modelInfo,configuration,trainMillis
0,items,{},"{'name': 'graphsage_demo', 'type': 'graphSage'}","{'maxIterations': 10, 'negativeSampleWeight': ...",27386


### **Add new data**

In [ ]:
query = """
UNWIND ['SEA OWL', 'LAND OWL', 'FIRE OWL', 'FOREST OWL', 'POTATO OWL'] as descr
MATCH (i:Item) where i.Description contains('SPACE OWL') WITH i, descr
CREATE (i2:Item{Description:descr})
CREATE (i) - [:COPURCHASED] -> (i2)
CREATE (i) <- [:COPURCHASED] - (i2);
"""

with driver.session(database="neo4j") as session:
    #display(session.run(query).consume().counters)
    result = session.run(query)
    results = pd.DataFrame([dict(record) for record in result])
results 

""


### **Drop and Load affinity graph again**

In [ ]:
query = """
CALL gds.graph.drop('items');
"""

with driver.session(database="neo4j") as session:
    #display(session.run(query).consume().counters)
    result = session.run(query)
    results = pd.DataFrame([dict(record) for record in result])
results

,graphName,database,memoryUsage,sizeInBytes,detailSizeInBytes,nodeProjection,relationshipProjection,nodeQuery,relationshipQuery,nodeCount,relationshipCount,density,creationTime,modificationTime,schema
0,items,neo4j,3063 KiB,3137496,"{'relationships': {'total': 2908528, 'everythi...",{'Item': {'properties': {'price_scaled': {'pro...,"{'copurchased': {'orientation': 'UNDIRECTED', ...",None,None,2864,2430254,0.296,2021-03-25T11:01:57.540516000-04:00,2021-03-25T11:01:59.243571000-04:00,"{'relationships': {'copurchased': {}}, 'nodes'..."


In [ ]:
query = """
CALL gds.graph.create(
  'items',
  {
    Item: {
      label: 'Item',
      properties: {
        price: {
          property: 'avg_price',
          defaultValue: 0.0
        },
        price_scaled: {
         property: 'avg_price_scaled',
         defaultValue: 0
       },
       total_sold: {
         property: 'num_sold',
         defaultValue: 0
       },
       total_sold_scaled: {
         property: 'num_sold_scaled',
         defaultValue: 0
       }
      }
    }
  },
  {
    copurchased: {
        type: 'COPURCHASED',
      orientation: 'UNDIRECTED'
    }
  }
);
"""

with driver.session(database="neo4j") as session:
    #display(session.run(query).consume().counters)
    result = session.run(query)
    results = pd.DataFrame([dict(record) for record in result])
results

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,createMillis
0,{'Item': {'properties': {'price_scaled': {'pro...,"{'copurchased': {'orientation': 'UNDIRECTED', ...",items,2869,2430274,154


### **Apply saved model to predict embeddings**

In [ ]:
query = """
CALL gds.beta.graphSage.write(
  'items',
  {
    writeProperty:'graphsage_embedding',
    modelName: 'graphsage_demo'
  });
"""

with driver.session(database="neo4j") as session:
    #display(session.run(query).consume().counters)
    result = session.run(query)
    results = pd.DataFrame([dict(record) for record in result])
results


,nodeCount,nodePropertiesWritten,createMillis,computeMillis,writeMillis,configuration
0,2869,2869,0,1814,95,"{'modelName': 'graphsage_demo', 'writeConcurre..."


### **Check embeddings for newly added data**

In [ ]:
query = """
UNWIND ['SEA OWL', 'LAND OWL', 'FIRE OWL', 'FOREST OWL', 'POTATO OWL'] as descr
MATCH (i:Item {Description:descr})
RETURN i.Description, i.graphsage_embedding;
"""

with driver.session(database="neo4j") as session:
    #display(session.run(query).consume().counters)
    result = session.run(query)
    results = pd.DataFrame([dict(record) for record in result])
results


,i.Description,i.graphsage_embedding
0,SEA OWL,"[1.2350249071117245e-08, 7.196271250569227e-11..."
1,LAND OWL,"[1.2350249071117245e-08, 7.196271250569227e-11..."
2,FIRE OWL,"[1.2350249071117245e-08, 7.196271250569227e-11..."
3,FOREST OWL,"[1.2350249071117245e-08, 7.196271250569227e-11..."
4,POTATO OWL,"[1.2350249071117245e-08, 7.196271250569227e-11..."


### **Calculate Similarity embedding**

In [ ]:
query = """
 MATCH (i:Item)
 WITH {item:id(i), weights: i.graphsage_embedding} AS itemData
 WITH collect(itemData) AS data
 CALL gds.alpha.similarity.cosine.write({
  data: data,
  skipValue: null,
  topK: 5,
  similarityCutoff:.1,
  writeRelationshipType:'SIMILAR_GRAPHSAGE_EMBEDDING'
 })
YIELD min, max, mean, stdDev, p25, p50, p75, p90, p95, p99, p999, p100
RETURN min, max, mean, stdDev;
"""

with driver.session(database="neo4j") as session:
    #display(session.run(query).consume().counters)
    result = session.run(query)
    results = pd.DataFrame([dict(record) for record in result])
results

,min,max,mean,stdDev
0,1.000,1.000,1.000,0.000


## **GraphSAGE** multipartite graph

In [ ]:
query = """
// Drop all graphs
CALL gds.graph.list()
YIELD graphName AS namedGraph
WITH namedGraph
CALL gds.graph.drop(namedGraph)
YIELD graphName
RETURN graphName;
"""

with driver.session(database="neo4j") as session:
    #display(session.run(query).consume().counters)
    result = session.run(query)
    results = pd.DataFrame([dict(record) for record in result])
results

""


In [ ]:
query = """
CALL gds.graph.create(
  'retail_graph',
  {
    Item: {
      label: 'Item',
      properties: {
        price: {
          property: 'avg_price',
          defaultValue: 0.0
        },
        price_scaled: {
          property: 'avg_price_scaled',
          defaultValue: 0.0
        },
        total_sold: {
         property: 'num_sold',
         defaultValue: 0
       },
       total_sold_scaled: {
         property: 'num_sold_scaled',
         defaultValue: 0
       }
     }
    },
    Transaction: {
      label: 'Transaction',
      properties: {
       EpochTime:{
       	property:'EpochTime',
        defaultValue:0
       }
     }
    }

 
 }, {
    
    CONTAINS: {
      type: 'CONTAINS',
      orientation: 'UNDIRECTED'
    }
});
"""

with driver.session(database="neo4j") as session:
    #display(session.run(query).consume().counters)
    result = session.run(query)
    results = pd.DataFrame([dict(record) for record in result])
results

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,createMillis
0,{'Transaction': {'properties': {'EpochTime': {...,"{'CONTAINS': {'orientation': 'UNDIRECTED', 'ag...",retail_graph,22718,944668,99


### Train a model

In [ ]:
query = """
// Drop all models, commumity edition, allows one model only
CALL gds.beta.model.list()
YIELD modelInfo AS mInfo
WITH mInfo.modelName as savedModel
CALL gds.beta.model.drop(savedModel)
YIELD modelInfo
RETURN modelInfo;
"""

with driver.session(database="neo4j") as session:
    #display(session.run(query).consume().counters)
    result = session.run(query)
    results = pd.DataFrame([dict(record) for record in result])
results

,modelInfo
0,"{'modelName': 'graphsage_multipartite_demo', '..."


In [ ]:
query = """
CALL gds.beta.graphSage.train(
  'retail_graph',
  {
    embeddingDimension: 8,
    modelName: 'graphsage_multipartite_demo',
    //which features should we use? all of them!
    featureProperties: ['price_scaled', 'EpochTime','total_sold_scaled'],
    //specify the projected feature dimension
    projectedFeatureDimension:5, //2 labels + 3 properties
    degreeAsProperty: true, //adding more properties
    epochs: 5, //how many times to traverse the graph during training
    searchDepth:5, //depth of the random walk,
    aggregator: 'mean', //aggregating data
    activationFunction: 'sigmoid', //part of the aggregation function
    sampleSizes:[25,10], //how many nodes to sample at each layer
    negativeSampleWeight: 10 //used by the loss function
  }
);
"""

with driver.session(database="neo4j") as session:
    #display(session.run(query).consume().counters)
    result = session.run(query)
    results = pd.DataFrame([dict(record) for record in result])
results

,graphName,graphCreateConfig,modelInfo,configuration,trainMillis
0,retail_graph,{},"{'name': 'graphsage_multipartite_demo', 'type'...","{'maxIterations': 10, 'negativeSampleWeight': ...",57279


### **Mutate in-memory graph**

In [ ]:
query = """
CALL gds.beta.graphSage.mutate(
  'retail_graph',
  {
    modelName: 'graphsage_multipartite_demo',
    mutateProperty:'Embedding'
  });
"""

with driver.session(database="neo4j") as session:
    #display(session.run(query).consume().counters)
    result = session.run(query)
    results = pd.DataFrame([dict(record) for record in result])
results

,nodePropertiesWritten,mutateMillis,nodeCount,createMillis,computeMillis,configuration
0,22718,1,22718,0,6093,"{'modelName': 'graphsage_multipartite_demo', '..."


### **Write Similarity Property to graph using KNN**

In [ ]:
query = """
CALL gds.beta.knn.write('retail_graph', {
    writeRelationshipType: 'SIMILAR_GRAPHSAGE_EMBEDDING_MULTIPARTITE',
    writeProperty: 'score',
    topK: 5,
    randomSeed: 42,
    nodeWeightProperty: 'Embedding'
})
YIELD nodesCompared, relationshipsWritten;
"""

with driver.session(database="neo4j") as session:
    #display(session.run(query).consume().counters)
    result = session.run(query)
    results = pd.DataFrame([dict(record) for record in result])
results

,nodesCompared,relationshipsWritten
0,22718,46164
